### Load preprocessed data & Setup Env

Run the script that downloads and processes the MovieLens data

In [15]:
!wget https://raw.githubusercontent.com/cemoody/simple_mf/master/src/download.py
!wget https://raw.githubusercontent.com/cemoody/simple_mf/master/notebooks/abstract_model.py

--2022-03-18 19:46:46--  https://raw.githubusercontent.com/cemoody/simple_mf/master/src/download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2721 (2.7K) [text/plain]
Saving to: ‘download.py.1’

download.py.1       100%[===================>]   2.66K  --.-KB/s    in 0s      

2022-03-18 19:46:46 (34.7 MB/s) - ‘download.py.1’ saved [2721/2721]

--2022-03-18 19:46:46--  https://raw.githubusercontent.com/cemoody/simple_mf/master/notebooks/abstract_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1696 (1.7K) [text/plain]


In [22]:
!pip install pytorch_lightning

     |████████████████████████████████| 527 kB 3.2 MB/s 
     |████████████████████████████████| 829 kB 49.0 MB/s 
     |████████████████████████████████| 134 kB 46.6 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
     |████████████████████████████████| 397 kB 53.2 MB/s 
     |████████████████████████████████| 952 kB 41.2 MB/s 
     |████████████████████████████████| 1.1 MB 38.8 MB/s 
     |████████████████████████████████| 271 kB 53.4 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 57.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=91dd325a0ea437bbe8f113517db0e04c4865b0a8eb18a43fb4c1b283068a6ca6
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successful

In [9]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 2.7 MB/s 
     |████████████████████████████████| 144 kB 53.1 MB/s 
     |████████████████████████████████| 181 kB 47.4 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=85f1fa170ec169412998bb3bf0421ec156f17fb9baed9d43488a96573e9a81cc
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [10]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
!python download.py

n_item 3953
n_user 6041
n_featuers 9994
n_occu 21
n_rows 1000209


##Transform data


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
X_test = fh['test_x'].astype(np.int64)
Y_test = fh['test_y']

X_train, X_val, Y_train, Y_val = train_test_split(train_x, train_y)


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(X_train)
print(' ')
print(Y_train)

[[2700 1732   24    7]
 [1667  157  112   16]
 [5493  780  311   12]
 ...
 [5585 2969  153    4]
 [4865 1251   28   14]
 [4447 2993  276    5]]
 
[[2.]
 [3.]
 [2.]
 ...
 [5.]
 [4.]
 [3.]]


In [2]:
X_train[:, 0].max()

6040

In [3]:
len(np.unique(X_train[:, 0]))

6040

In [4]:
Y_train

array([[2.],
       [3.],
       [2.],
       ...,
       [5.],
       [4.],
       [3.]], dtype=float32)

### Define data loaders

In [5]:
from torch import from_numpy
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import BatchSampler
from torch.utils.data import RandomSampler


def dataloader(*arrs, batch_size=32):
    dataset = TensorDataset(*arrs)
    bs = BatchSampler(RandomSampler(dataset), 
                      batch_size=batch_size, drop_last=False)
    return DataLoader(dataset, batch_sampler=bs, num_workers=8)
 
train = dataloader(from_numpy(X_train), from_numpy(Y_train))
test = dataloader(from_numpy(X_test), from_numpy(Y_test))
val = dataloader(from_numpy(X_val), from_numpy(Y_val))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Define the MF Model

In [6]:
from abstract_model import AbstractModel

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, batch_size=128):
        super().__init__()
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.batch_size = batch_size
        self.save_hyperparameters()
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # equivalent:
        # self.user.weight[user_id, :]
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        # shape vector_user is (batch_size, k)
        # vector_user * vector_item is shape (batch_size, k)
        # sum(vector_user * vector_item is shape, dim=1) (batch_size)
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        return ui_interaction
    
    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        # target is (batchsize, 1)
        # target.squeeze (batchsize, )
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse, {"mse": loss_mse}
    
    def reg(self):
        # Compute L2 reularization over user (P) and item (Q) matrices
        reg_user =  l2_regularize(self.user.weight) * self.c_vector
        reg_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        log = {"reg_user": reg_user, "reg_item": reg_item}
        total = reg_user + reg_item
        return total, log

Note, I typically tune the initial batch size to get an efficient number of datapoints observed per second. On a CPU run `htop` or `top` in another window / terminal tab to see how efficiently your CPU is being  used. If it spends a lot of time below 50% utilization, crank up  the batch size. GPUs are more sensitive, and it's difficult to keep them well fed. To measure efficiency, use `nvidia-smi`. That command only  gets you a snapshot though, so to keep it running use `watch -n 0.1 nvidia-smi`.

In [8]:
from pytorch_lightning.loggers.wandb import WandbLogger

batch_size = 1024
k = 5
c_vector = 1e-5
model = MF(n_user, n_item, k=k, c_vector=c_vector,
          batch_size=batch_size)

# add a logger
logger = WandbLogger(name="01_mf", project="simple_mf")

trainer = pl.Trainer(max_epochs=100, logger=logger,
                     gpus=0,
                     log_every_n_steps=1,
                     progress_bar_refresh_rate=1) 


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model, train, val)

wandb: Currently logged in as: chrisemoody (use `wandb login --relogin` to force relogin)



  | Name | Type      | Params
-----------------------------------
0 | user | Embedding | 30.2 K
1 | item | Embedding | 19.8 K
-----------------------------------
50.0 K    Trainable params
0         Non-trainable params
50.0 K    Total params
0.200     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/optimization/closure.py:36: LightningDeprecationWarning: One of the returned values {'train_loss', 'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


In [ ]:
results = trainer.test(model)
mse = results['avg_test_loss']
rmse = np.sqrt(mse)
rmse